# Full audio

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COSWARA - HYBRID MODEL FULL AUDIO')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
n_mfcc = 13

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'

meta_coswara = os.path.join(root, 'full_audio/coswara_metadata_for_label_matching.csv')
meta_coughvid = os.path.join(root, 'full_audio/coughvid_metadata_for_label_matching.csv')


mfcc_coswara = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coswara_features.npy')
mfcc_coughvid_1 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_20k.npy')
mfcc_coughvid_2 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_20k_40k.npy')
mfcc_coughvid_3 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_40k_60k.npy')
mfcc_coughvid_4 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_60k_75k.npy')
mfcc_coughvid_5 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_9981.npy')


### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coswara)
meta_df.head()

,id,age,covid_status,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,...,is_fatigue,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,cough_path,label,error
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,28,healthy,2020-04-23,y,male,India,Anantapur,Andhra Pradesh,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,healthy,0
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,25,healthy,2020-04-20,y,male,India,BENGALURU URBAN,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/AxuYWBN0jFVLINCBqIW5aZmGCdu1/cough-shallow.wav,healthy,0
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,28,healthy,2020-04-24,y,female,United States,Pittsburgh,Pennsylvania,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/C5eIsssb9GSkaAgIfsHMHeR6fSh1/cough-shallow.wav,healthy,0
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,26,healthy,2020-04-23,y,male,India,Bangalore,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/YjbEAECMBIaZKyfqOvWy5DDImUb2/cough-shallow.wav,healthy,0
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,32,healthy,2020-04-22,y,male,India,Nalanda,Bihar,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/aGOvk4ji0cVqIzCs1jHnzlw2UEy2/cough-shallow.wav,healthy,0


In [ ]:
logger.info(meta_df.shape)

2022-04-28 14:37:45,843 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4465, 39)


In [ ]:
mfcc_feature = np.load(mfcc_coswara)
meta_df = pd.read_csv(meta_coswara)
meta_df = meta_df[meta_df['error'] == 0].reset_index(drop=True)

In [ ]:
logger.info(meta_df.shape)

2022-04-28 14:37:46,913 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4400, 39)


In [ ]:
meta_df.label.value_counts().sum()

4400

In [ ]:
mfcc_feature[0].shape, len(mfcc_feature), meta_df.shape

((234, 246), 4400, (4400, 39))

In [ ]:
DROP_COLS = [
             'cough_path',
             'error',
             'record_date',
             'id',
             'covid_status',
             'test_date'
]

meta_df.isna().sum()

id                                  0
age                                 0
covid_status                        0
record_date                         0
is_english_proficiency              0
gender                              0
country                             0
locality                          471
state                               0
is_returning_user                1335
is_smoker                        3273
is_cold                          3825
is_hypertension                  4042
is_diabetes                      4024
is_cough                         3638
date_of_ct_scan                  4135
has_ctScan                       3510
ct_score                         4135
is_diarrheoa                     4332
is_fever                         3965
is_loss_of_smell                 4153
is_muscle_pain                   4031
test_type                        3657
test_date                        3657
test_status                      2772
is_using_mask                    2040
vaccination_

In [ ]:
# fill na with unknown value
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

id                               0
age                              0
covid_status                     0
record_date                      0
is_english_proficiency           0
gender                           0
country                          0
locality                         0
state                            0
is_returning_user                0
is_smoker                        0
is_cold                          0
is_hypertension                  0
is_diabetes                      0
is_cough                         0
date_of_ct_scan                  0
has_ctScan                       0
ct_score                         0
is_diarrheoa                     0
is_fever                         0
is_loss_of_smell                 0
is_muscle_pain                   0
test_type                        0
test_date                        0
test_status                      0
is_using_mask                    0
vaccination_status               0
is_breathing_difficulty          0
is_others_resp      

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)
logger.info(meta_df.shape)

2022-04-28 14:37:46,987 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4400, 33)


In [ ]:
# replace label with binary numbers
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'positive_mild': 1,
    'no_resp_illness_exposed': 1,
    'resp_illness_not_identified': 0,
    'positive_moderate': 1,
    'recovered_full': 0,
    'positive_asymp': 1
})
meta_df.label.value_counts()

0    3194
1    1206
Name: label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label', 'age'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [ ]:
mfcc_feature = np.load(mfcc_coswara)

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-28 14:37:48,892 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - 1206, 3194


In [ ]:
# get features for each class
feature_0 = mfcc_feature[index_0]
feature_1 = mfcc_feature[index_1]

In [ ]:
print(len(feature_1))

1206


In [ ]:
del mfcc_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:2555], feature_1[:965]), axis=0)
  val = np.concatenate((feature_0[2555:2875], feature_1[965:1086]), axis=0)
  test = np.concatenate((feature_0[2875:], feature_1[1086:]), axis=0)

  y_train = pd.concat([label0[:2555], label1[:965]], ignore_index=True)
  y_val = pd.concat([label0[2555:2875], label1[965:1086]], ignore_index=True)
  y_test = pd.concat([label0[2875:], label1[1086:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(3520, 234, 246)


In [ ]:
y_train_2d

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(3520, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(3520, 32, 1)

In [ ]:
X_train_1d.shape

(3520, 32, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(merged_model, to_file='model_final.png')
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
from datetime import datetime
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)


Epoch 1/40
28/28 [==============================] - 22s 758ms/step - loss: 0.0544 - auc: 0.9964 - val_loss: 2.1957 - val_auc: 0.7054
Epoch 2/40
28/28 [==============================] - 21s 752ms/step - loss: 0.0308 - auc: 0.9985 - val_loss: 2.3608 - val_auc: 0.7068
Epoch 3/40
28/28 [==============================] - 21s 749ms/step - loss: 0.0228 - auc: 0.9989 - val_loss: 2.3612 - val_auc: 0.6768
Epoch 4/40
28/28 [==============================] - 21s 754ms/step - loss: 0.0265 - auc: 0.9988 - val_loss: 2.7846 - val_auc: 0.6823
Epoch 5/40
28/28 [==============================] - 21s 758ms/step - loss: 0.0258 - auc: 0.9988 - val_loss: 2.6948 - val_auc: 0.6452
Epoch 6/40
28/28 [==============================] - 21s 750ms/step - loss: 0.0237 - auc: 0.9989 - val_loss: 2.8616 - val_auc: 0.6915
Epoch 7/40
28/28 [==============================] - 21s 749ms/step - loss: 0.0292 - auc: 0.9982 - val_loss: 2.7528 - val_auc: 0.6520


In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

14/14 [==============================] - 0s 28ms/step - loss: 2.4708 - auc: 0.6626


[2.47084903717041, 0.6625925302505493]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_13_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(3520, 234, 246)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(3520, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(3520, 32, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([3520, 2])

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
LEARNING_RATE = 0.001
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5


opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_melme_hybrid_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

## SMOTE

In [ ]:
X_train_2d.shape

(3520, 234, 246)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(5110, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(5110, 32, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([5110, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_hybrid_mel_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

Epoch 1/40
639/639 [==============================] - 54s 70ms/step - loss: 0.6909 - auc: 0.6618 - val_loss: 1.4674 - val_auc: 0.3583
Epoch 2/40
639/639 [==============================] - 43s 68ms/step - loss: 0.5676 - auc: 0.7813 - val_loss: 0.6356 - val_auc: 0.7530
Epoch 3/40
639/639 [==============================] - 42s 66ms/step - loss: 0.5526 - auc: 0.7956 - val_loss: 0.7789 - val_auc: 0.7270
Epoch 4/40
639/639 [==============================] - 42s 66ms/step - loss: 0.5365 - auc: 0.8109 - val_loss: 2.5594 - val_auc: 0.5591
Epoch 5/40
639/639 [==============================] - 43s 67ms/step - loss: 0.5116 - auc: 0.8304 - val_loss: 0.6904 - val_auc: 0.7731
Epoch 6/40
639/639 [==============================] - 43s 67ms/step - loss: 0.4981 - auc: 0.8405 - val_loss: 0.5793 - val_auc: 0.7711
Epoch 7/40
639/639 [==============================] - 43s 67ms/step - loss: 0.4971 - auc: 0.8405 - val_loss: 0.7823 - val_auc: 0.7777
Epoch 8/40
639/639 [==============================] - 42s 66ms

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_13_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_13_hybrid_model_40_8_0.001/assets


Epoch 1/40
320/320 [==============================] - 41s 125ms/step - loss: 0.2562 - auc: 0.9565 - val_loss: 0.7092 - val_auc: 0.7058
Epoch 2/40
320/320 [==============================] - 39s 122ms/step - loss: 0.2131 - auc: 0.9689 - val_loss: 0.8237 - val_auc: 0.7464
Epoch 3/40
320/320 [==============================] - 40s 124ms/step - loss: 0.1786 - auc: 0.9778 - val_loss: 1.4577 - val_auc: 0.5919
Epoch 4/40
320/320 [==============================] - 39s 123ms/step - loss: 0.1536 - auc: 0.9824 - val_loss: 1.0234 - val_auc: 0.7558
Epoch 5/40
320/320 [==============================] - 39s 123ms/step - loss: 0.1490 - auc: 0.9821 - val_loss: 1.2027 - val_auc: 0.6644
Epoch 6/40
320/320 [==============================] - 39s 123ms/step - loss: 0.1210 - auc: 0.9870 - val_loss: 1.6603 - val_auc: 0.7093
Epoch 7/40
320/320 [==============================] - 40s 124ms/step - loss: 0.1126 - auc: 0.9884 - val_loss: 1.2844 - val_auc: 0.7206
Epoch 8/40
320/320 [==============================] - 3

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_13_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_13_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_13_hybrid_model_40_64_0.001/assets


Epoch 1/40
40/40 [==============================] - 42s 978ms/step - loss: 0.0135 - auc: 0.9999 - val_loss: 2.7041 - val_auc: 0.7551
Epoch 2/40
40/40 [==============================] - 36s 905ms/step - loss: 0.0133 - auc: 0.9999 - val_loss: 2.7460 - val_auc: 0.7578
Epoch 3/40
40/40 [==============================] - 36s 905ms/step - loss: 0.0138 - auc: 0.9998 - val_loss: 2.7497 - val_auc: 0.7574
Epoch 4/40
40/40 [==============================] - 36s 909ms/step - loss: 0.0135 - auc: 0.9999 - val_loss: 2.7443 - val_auc: 0.7589
Epoch 5/40
40/40 [==============================] - 36s 909ms/step - loss: 0.0131 - auc: 0.9999 - val_loss: 2.7552 - val_auc: 0.7583
Epoch 6/40
40/40 [==============================] - 36s 913ms/step - loss: 0.0135 - auc: 0.9999 - val_loss: 2.7616 - val_auc: 0.7563
Epoch 7/40
40/40 [==============================] - 36s 913ms/step - loss: 0.0130 - auc: 0.9999 - val_loss: 2.7715 - val_auc: 0.7589
Epoch 8/40
40/40 [==============================] - 36s 912ms/step - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_13_hybrid_model_40_128_0.001/assets


## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 4s 65ms/step - loss: 5.4899 - auc: 0.4871


[5.489938735961914, 0.48708757758140564]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 2s 43ms/step - loss: 1.8304 - auc: 0.7579


[1.8303685188293457, 0.7578935027122498]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 2s 43ms/step - loss: 0.8631 - auc: 0.6726


[0.8631494641304016, 0.672568142414093]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 2s 45ms/step - loss: 0.5246 - auc: 0.8224


[0.5246315598487854, 0.8224480152130127]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 2s 43ms/step - loss: 0.5571 - auc: 0.7930


[0.5571303367614746, 0.7929934859275818]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 44ms/step - loss: 2.4708 - auc: 0.6626


[2.47084903717041, 0.6625925302505493]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 45ms/step - loss: 1.8100 - auc: 0.7329


[1.8099826574325562, 0.7328625917434692]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 46ms/step - loss: 0.8673 - auc: 0.6615


[0.8672693371772766, 0.6615133285522461]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 43ms/step - loss: 0.5468 - auc: 0.8361


[0.5467538833618164, 0.8360635042190552]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 44ms/step - loss: 0.5385 - auc: 0.8133


[0.5384811162948608, 0.8133260011672974]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 44ms/step - loss: 2.6709 - auc: 0.7604


[2.6709182262420654, 0.7603737711906433]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 45ms/step - loss: 2.5517 - auc: 0.7543


[2.55169939994812, 0.7543029189109802]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 44ms/step - loss: 2.2642 - auc: 0.7574


[2.264206647872925, 0.7574369311332703]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 43ms/step - loss: 1.0950 - auc: 0.7785


[1.0949925184249878, 0.7785452008247375]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 44ms/step - loss: 1.5729 - auc: 0.5660


[1.572911262512207, 0.5659632086753845]

# Chunks
* potentials: use `num_chunk` to explore as long audio influence the model'accuracy (the more `num_chunk` the longer the audio)

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COSWARA - HYBRID MODEL CHUNKS')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
n_mfcc = 13

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing'

meta_coswara = os.path.join(data_root, 'cough_chunking/coswara/coswara_chunk_metadata.csv')
meta_coughvid = os.path.join(data_root, 'cough_chunking/coughvid/coughvid_chunk_metadata.csv')

mel_coswara = os.path.join(root, 'chunks/melspectrogram_coswara_features.npy')
mel_coughvid_1 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_20k.npy')
mel_coughvid_2 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_20k_40k.npy')
mel_coughvid_3 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_40k_60k.npy')
mel_coughvid_4 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_60k_75k.npy')
mel_coughvid_5 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_9981.npy')

mfcc_coswara = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coswara_features.npy')
mfcc_coughvid_1 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k.npy')
mfcc_coughvid_2 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k_40k.npy')
mfcc_coughvid_3 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_40k_60k.npy')
mfcc_coughvid_4 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_60k_75k.npy')
mfcc_coughvid_5 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_9981.npy')


### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coswara)
meta_df.head()

,uuid,age,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,is_smoker,...,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,label,sr_cough,chunk_path,num_chunk
0,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0.wav,8
1,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1.wav,8
2,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2.wav,8
3,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3.wav,8
4,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4.wav,8


In [ ]:
logger.info(meta_df.shape)

2022-04-26 14:03:28,638 - COSWARA - HYBRID MODEL CHUNKS - INFO - (22878, 39)


In [ ]:
DROP_COLS = [
             'sr_cough',
             'chunk_path',
             'record_date',
             'uuid',
             'test_date'
]

meta_df.isna().sum()

uuid                                 0
age                                  0
record_date                          0
is_english_proficiency               0
gender                               0
country                              0
locality                          2295
state                                0
is_returning_user                 7281
is_smoker                        17592
is_cold                          20117
is_hypertension                  21030
is_diabetes                      20985
is_cough                         19040
date_of_ct_scan                  21754
has_ctScan                       18880
ct_score                         21754
is_diarrheoa                     22514
is_fever                         20704
is_loss_of_smell                 21621
is_muscle_pain                   21229
test_type                        19560
test_date                        19560
test_status                      14906
is_using_mask                    11268
vaccination_status       

In [ ]:
# fill na with unknown value
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                             0
age                              0
record_date                      0
is_english_proficiency           0
gender                           0
country                          0
locality                         0
state                            0
is_returning_user                0
is_smoker                        0
is_cold                          0
is_hypertension                  0
is_diabetes                      0
is_cough                         0
date_of_ct_scan                  0
has_ctScan                       0
ct_score                         0
is_diarrheoa                     0
is_fever                         0
is_loss_of_smell                 0
is_muscle_pain                   0
test_type                        0
test_date                        0
test_status                      0
is_using_mask                    0
vaccination_status               0
is_breathing_difficulty          0
is_others_resp                   0
is_fatigue          

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)
logger.info(meta_df.shape)

2022-04-26 14:03:28,786 - COSWARA - HYBRID MODEL CHUNKS - INFO - (22878, 34)


In [ ]:
# replace label with binary numbers
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'positive_mild': 1,
    'no_resp_illness_exposed': 1,
    'resp_illness_not_identified': 0,
    'positive_moderate': 1,
    'recovered_full': 0,
    'positive_asymp': 1
})
meta_df.label.value_counts()

0    16871
1     6007
Name: label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label', 'age'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [ ]:
mfcc_feature = np.load(mfcc_coswara)

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-26 14:03:47,429 - COSWARA - HYBRID MODEL CHUNKS - INFO - 6007, 16871


In [ ]:
# get features for each class
feature_0 = mfcc_feature[index_0]
feature_1 = mfcc_feature[index_1]

In [ ]:
print(len(feature_1))

6007


In [ ]:
del mfcc_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:13504], feature_1[:4803]), axis=0)
  val = np.concatenate((feature_0[13504:15186], feature_1[4803:5408]), axis=0)
  test = np.concatenate((feature_0[15186:], feature_1[5408:]), axis=0)

  y_train = pd.concat([label0[:13504], label1[:4803]], ignore_index=True)
  y_val = pd.concat([label0[13504:15186], label1[4803:5408]], ignore_index=True)
  y_test = pd.concat([label0[15186:], label1[5408:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(18307, 234, 236)


In [ ]:
y_train_2d

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(18307, 234, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(18307, 33, 1)

In [ ]:
X_train_1d.shape

(18307, 33, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(merged_model, to_file='model_final.png')
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
144/144 [==============================] - 99s 681ms/step - loss: 0.0049 - auc: 0.9998 - val_loss: 1.4882 - val_auc: 0.8328
Epoch 2/40
144/144 [==============================] - 98s 677ms/step - loss: 0.0021 - auc: 0.9999 - val_loss: 1.5822 - val_auc: 0.8198
Epoch 3/40
144/144 [==============================] - 97s 672ms/step - loss: 0.0773 - auc: 0.9923 - val_loss: 1.0813 - val_auc: 0.8095
Epoch 4/40
144/144 [==============================] - 97s 674ms/step - loss: 0.0085 - auc: 0.9998 - val_loss: 1.1615 - val_auc: 0.8301
Epoch 5/40
144/144 [==============================] - 97s 672ms/step - loss: 0.0688 - auc: 0.9929 - val_loss: 1.0811 - val_auc: 0.8347
Epoch 6/40
144/144 [==============================] - 97s 672ms/step - loss: 0.0465 - auc: 0.9965 - val_loss: 1.1948 - val_auc: 0.8266
Epoch 7/40
144/144 [==============================] - 98s 679ms/step - loss: 0.0105 - auc: 0.9996 - val_loss: 1.2785 - val_auc: 0.8258
Epoch 8/40
144/144 [==============================] - 9

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

72/72 [==============================] - 2s 28ms/step - loss: 2.2329 - auc: 0.7494


[2.2329044342041016, 0.749441921710968]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coswara_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coswara_mfcc_13_hybrid_model_40_128_0.001/assets


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

## Standard scaler model

In [ ]:
X_train_2d.shape

(18307, 234, 236)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(18307, 234, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(18307, 33, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([18307, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, f'/coswara_mfcc_{n_mfcc}_hybrid_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
144/144 [==============================] - 148s 890ms/step - loss: 0.6429 - auc: 0.7457 - val_loss: 0.5677 - val_auc: 0.8284
Epoch 2/40
144/144 [==============================] - 124s 863ms/step - loss: 0.5480 - auc: 0.7993 - val_loss: 0.5775 - val_auc: 0.7715
Epoch 3/40
144/144 [==============================] - 124s 861ms/step - loss: 0.5332 - auc: 0.8132 - val_loss: 0.6371 - val_auc: 0.6972
Epoch 4/40
144/144 [==============================] - 124s 859ms/step - loss: 0.5454 - auc: 0.8017 - val_loss: 0.5848 - val_auc: 0.7621
Epoch 5/40
144/144 [==============================] - 124s 863ms/step - loss: 0.5314 - auc: 0.8120 - val_loss: 0.6401 - val_auc: 0.7465
Epoch 6/40
144/144 [==============================] - 124s 862ms/step - loss: 0.5029 - auc: 0.8352 - val_loss: 0.6213 - val_auc: 0.7727


In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

72/72 [==============================] - 3s 42ms/step - loss: 0.5739 - auc: 0.8028


[0.573883593082428, 0.8027680516242981]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'std_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coswara_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coswara_mfcc_13_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(18307, 234, 236)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(27008, 234, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(27008, 33, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([27008, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, f'/coswara_hybrid_mfcc_{n_mfcc}_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
211/211 [==============================] - 185s 865ms/step - loss: 0.0065 - auc: 0.9997 - val_loss: 2.4142 - val_auc: 0.7407
Epoch 2/40
211/211 [==============================] - 181s 859ms/step - loss: 9.3107e-04 - auc: 1.0000 - val_loss: 2.4444 - val_auc: 0.7614
Epoch 3/40
211/211 [==============================] - 178s 844ms/step - loss: 4.3088e-04 - auc: 1.0000 - val_loss: 2.7065 - val_auc: 0.7566
Epoch 4/40
211/211 [==============================] - 179s 847ms/step - loss: 1.8787e-04 - auc: 1.0000 - val_loss: 2.8371 - val_auc: 0.7556
Epoch 5/40
211/211 [==============================] - 179s 850ms/step - loss: 1.2105e-04 - auc: 1.0000 - val_loss: 2.8786 - val_auc: 0.7549
Epoch 6/40
211/211 [==============================] - 178s 845ms/step - loss: 1.1953e-04 - auc: 1.0000 - val_loss: 3.0323 - val_auc: 0.7459
Epoch 7/40
211/211 [==============================] - 180s 853ms/step - loss: 1.1325e-04 - auc: 1.0000 - val_loss: 3.1194 - val_auc: 0.7448


In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d], verbose=1)

72/72 [==============================] - 3s 38ms/step - loss: 3.0836 - auc: 0.7302


[3.083585262298584, 0.730211079120636]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coswara_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coswara_mfcc_13_hybrid_model_40_128_0.001/assets


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs/fit

## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 12s 41ms/step - loss: 0.5739 - auc: 0.8028


[0.5738835334777832, 0.8027680516242981]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 4s 38ms/step - loss: 0.7100 - auc: 0.7291


[0.7100050449371338, 0.7290571928024292]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 4s 38ms/step - loss: 0.5846 - auc: 0.8200


[0.5845623016357422, 0.819994330406189]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 4s 38ms/step - loss: 0.8960 - auc: 0.8010


[0.8960346579551697, 0.8009706735610962]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 4s 38ms/step - loss: 0.5769 - auc: 0.7828


[0.5769414901733398, 0.7827572226524353]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 39ms/step - loss: 2.2329 - auc: 0.7494


[2.2329039573669434, 0.749441921710968]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 39ms/step - loss: 2.1375 - auc: 0.7590


[2.137491226196289, 0.7590426206588745]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 39ms/step - loss: 1.1882 - auc: 0.7810


[1.1881636381149292, 0.7809679508209229]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 39ms/step - loss: 0.8016 - auc: 0.8013


[0.8016014695167542, 0.801252543926239]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 39ms/step - loss: 0.6173 - auc: 0.7983


[0.6173203587532043, 0.7983224391937256]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 39ms/step - loss: 3.0836 - auc: 0.7302


[3.083585023880005, 0.730211079120636]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 39ms/step - loss: 2.0345 - auc: 0.6990


[2.0345308780670166, 0.6990023851394653]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 39ms/step - loss: 1.6747 - auc: 0.7410


[1.6747167110443115, 0.7409906387329102]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 40ms/step - loss: 1.4605 - auc: 0.7711


[1.4604740142822266, 0.7711302638053894]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 39ms/step - loss: 0.7622 - auc: 0.7766


[0.762173593044281, 0.7765684723854065]